In [70]:
import joblib

# Define your pipeline (assuming it is already defined as `pipeline`)
# Save the pipeline to a file
joblib.dump(pipeline, 'assets/combined_pipeline.pkl')

['assets/combined_pipeline.pkl']

In [76]:
import joblib

# Load the pipeline from the file
loaded_pipeline = joblib.load('assets/combined_pipeline.pkl')

# Use the loaded pipeline for predictions
y_class_pred, y_reg_pred = loaded_pipeline.predict(X_test)

In [72]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate classification model
accuracy = accuracy_score(y_class_test, y_class_pred)
precision = precision_score(y_class_test, y_class_pred)
recall = recall_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred)

print(f"Classification Model - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

# Filter test data based on classification predictions for regression evaluation
X_test_reg = X_test[y_class_pred == 1]
y_reg_test_filtered = y_reg_test[y_class_pred == 1]

# Evaluate regression model
mae = mean_absolute_error(y_reg_test_filtered, y_reg_pred)
mse = mean_squared_error(y_reg_test_filtered, y_reg_pred)
r2 = r2_score(y_reg_test_filtered, y_reg_pred)

print(f"Regression Model - MAE: {mae}, MSE: {mse}, R2: {r2}")

Classification Model - Accuracy: 0.9998206599713056, Precision: 1.0, Recall: 0.9983525535420099, F1-Score: 0.9991755976916735
Regression Model - MAE: 15610.609491043551, MSE: 907077462.8011646, R2: 0.6691194156944285


In [25]:
df = pd.read_csv('assets/prosperLoanData.csv')
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),BorrowerState,Occupation,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentlyInGroup,GroupKey,DateCreditPulled,CreditScoreRangeLower,CreditScoreRangeUpper,FirstRecordedCreditLine,CurrentCreditLines,OpenCreditLines,TotalCreditLinespast7years,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,InquiriesLast6Months,TotalInquiries,CurrentDelinquencies,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,PublicRecordsLast12Months,...,BankcardUtilization,AvailableBankcardCredit,TotalTrades,TradesNeverDelinquent (percentage),TradesOpenedLast6Months,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome,LoanKey,TotalProsperLoans,TotalProsperPaymentsBilled,OnTimeProsperPayments,ProsperPaymentsLessThanOneMonthLate,ProsperPaymentsOneMonthPlusLate,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,ScorexChangeAtTimeOfListing,LoanCurrentDaysDelinquent,LoanFirstDefaultedCycleNumber,LoanMonthsSinceOrigination,LoanNumber,LoanOriginalAmount,LoanOriginationDate,LoanOriginationQuarter,MemberKey,MonthlyLoanPayment,LP_CustomerPayments,LP_CustomerPrincipalPayments,LP_InterestandFees,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,NaN,NaN,NaN,NaN,NaN,NaN,0,CO,Other,Self-employed,2.0,True,True,NaN,2007-08-26 18:41:46.780000000,640.0,659.0,2001-10-11 00:00:00,5.0,4.0,12.0,1,24.0,3.0,3.0,2.0,472.0,4.0,0.0,0.0,...,0.00,1500.0,11.0,0.81,0.0,0.17,"$25,000-49,999",True,3083.333333,E33A3400205839220442E84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,78,19141,9425,2007-09-12 00:00:00,Q3 2007,1F3E3376408759268057EDA,330.43,11396.14,9425.00,1971.14,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,0.07960,0.0249,0.05470,6.0,A,7.0,2,CO,Professional,Employed,44.0,False,False,NaN,2014-02-27 08:28:14,680.0,699.0,1996-03-18 00:00:00,14.0,14.0,29.0,13,389.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,...,0.21,10266.0,29.0,1.00,2.0,0.18,"$50,000-74,999",True,6125.000000,9E3B37071505919926B1D82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,134815,10000,2014-03-03 00:00:00,Q1 2014,1D13370546739025387B2F4,318.93,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,NaN,NaN,NaN,NaN,NaN,NaN,0,GA,Other,Not available,NaN,False,True,783C3371218786870A73D20,2007-01-02 14:09:10.060000000,480.0,499.0,2002-07-27 00:00:00,NaN,NaN,3.0,0,0.0,0.0,1.0,1.0,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.06,Not displayed,True,2083.333333,6954337960046817851BCB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,86,6466,3001,2007-01-17 00:00:00,Q1 2007,5F7033715035555618FA612,123.32,4186.63,3001.00,1185.63,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,0.08490,0.0249,0.06000,6.0,A,9.0,16,GA,Skilled Labor,Employed,113.0,True,False,NaN,2012-10-22 11:02:32,800.0,819.0,1983-02-28 00:00:00,5.0,5.0,29.0,7,115.0,0.0,1.0,4.0,10056.0,14.0,0.0,0.0,...,0.04,30754.0,26.0,0.76,0.0,0.15,"$25,000-49,999",True,2875.000000,A0393664465886295619C51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,16,77296,10000,2012-11-01 00:00:00,Q4 2012,9ADE356069835475068C6D2,321.45,5143.20,4091.09,1052.11,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,0.18316,0.0925,0.09066,3.0,D,4.0,2,MN,Executive,Employed,44.0,

In [27]:
col = ['ListingNumber',
'BorrowerAPR',
'BorrowerRate', 
'ClosedDate',
'LoanMonthsSinceOrigination', 
'LoanNumber' ,
'LoanOriginationQuarter',
'LP_CustomerPayments' ,
'LP_CustomerPrincipalPayments',
'LoanCurrentDaysDelinquent' ,
'MonthlyLoanPayment' ,
'EmploymentStatus']

df[col]

,ListingNumber,BorrowerAPR,BorrowerRate,ClosedDate,LoanMonthsSinceOrigination,LoanNumber,LoanOriginationQuarter,LP_CustomerPayments,LP_CustomerPrincipalPayments,LoanCurrentDaysDelinquent,MonthlyLoanPayment,EmploymentStatus
0,193129,0.16516,0.1580,2009-08-14 00:00:00,78,19141,Q3 2007,11396.1400,9425.00,0,330.43,Self-employed
1,1209647,0.12016,0.0920,NaN,0,134815,Q1 2014,0.0000,0.00,0,318.93,Employed
2,81716,0.28269,0.2750,2009-12-17 00:00:00,86,6466,Q1 2007,4186.6300,3001.00,0,123.32,Not available
3,658116,0.12528,0.0974,NaN,16,77296,Q4 2012,5143.2000,4091.09,0,321.45,Employed
4,909464,0.24614,0.2085,NaN,6,102670,Q3 2013,2819.8500,1563.22,0,563.97,Employed
...,...,...,...,...,...,...,...,...,...,...,...,...
113932,753087,0.22354,0.1864,NaN,11,88485,Q2 2013,3647.4000,2238.38,0,364.74,Employed
113933,537216,0.13220,0.1110,NaN,28,55801,Q4 2011,2330.5500,1997.16,0,65.57,Employed
113934,1069178,0.23984,0.2150,NaN,3,123122,Q4 2013,546.7000,183.15,0,273.35,Employed
113935,539056,0.28408,0.2605,2013-08-13 00:00:00,28,56401,Q4 2011,21122.5600,15000.00,0,449.55,Full-time


In [51]:
df_1 = pd.read_csv('assets/model_data.csv')

In [52]:
df_1.head()

,LoanStatus,ELA,ListingNumber,BorrowerAPR,BorrowerRate,LoanMonthsSinceOrigination,LoanNumber,LoanOriginationQuarter,LP_CustomerPayments,LP_CustomerPrincipalPayments,LoanCurrentDaysDelinquent,MonthlyLoanPayment,EmploymentStatus,date_numeric
0,0,63034.0,193129,0.16516,0.1580,78,19141,Q3 2007,11396.14,9425.00,0,330.43,Self-employed,3513.0
1,0,43120.0,1209647,0.12016,0.0920,0,134815,Q1 2014,0.00,0.00,0,318.93,Employed,NaN
2,0,45064.0,658116,0.12528,0.0974,16,77296,Q4 2012,5143.20,4091.09,0,321.45,Employed,NaN
3,0,127590.0,909464,0.24614,0.2085,6,102670,Q3 2013,2819.85,1563.22,0,563.97,Employed,NaN
4,0,133260.0,1074836,0.15425,0.1314,3,123257,Q4 2013,679.34,351.89,0,342.37,Employed,NaN


In [53]:
df_1['ClosedDate'].value_counts()

KeyError: 'ClosedDate'

In [41]:
df_1['ClosedDate'] = pd.to_datetime(df_1['ClosedDate'])

In [42]:
reference_date = pd.Timestamp('2000-01-01')

# Convert 'ClosedDate' to numeric format
df_1['date_numeric'] = (df_1['ClosedDate'] - reference_date).dt.days

In [44]:
df_1.head()

,LoanStatus,ELA,ListingNumber,BorrowerAPR,BorrowerRate,ClosedDate,LoanMonthsSinceOrigination,LoanNumber,LoanOriginationQuarter,LP_CustomerPayments,LP_CustomerPrincipalPayments,LoanCurrentDaysDelinquent,MonthlyLoanPayment,EmploymentStatus,date_numeric
0,0,63034.0,193129,0.16516,0.1580,2009-08-14,78,19141,Q3 2007,11396.14,9425.00,0,330.43,Self-employed,3513.0
1,0,43120.0,1209647,0.12016,0.0920,NaT,0,134815,Q1 2014,0.00,0.00,0,318.93,Employed,NaN
2,0,45064.0,658116,0.12528,0.0974,NaT,16,77296,Q4 2012,5143.20,4091.09,0,321.45,Employed,NaN
3,0,127590.0,909464,0.24614,0.2085,NaT,6,102670,Q3 2013,2819.85,1563.22,0,563.97,Employed,NaN
4,0,133260.0,1074836,0.15425,0.1314,NaT,3,123257,Q4 2013,679.34,351.89,0,342.37,Employed,NaN


In [46]:
df_1.drop(columns=['ClosedDate'], inplace=True)

In [47]:
df_1.to_csv('assets/model_data.csv', index=False)

In [54]:
df_1.isna().sum()

LoanStatus                          0
ELA                                 0
ListingNumber                       0
BorrowerAPR                         0
BorrowerRate                        0
LoanMonthsSinceOrigination          0
LoanNumber                          0
LoanOriginationQuarter              0
LP_CustomerPayments                 0
LP_CustomerPrincipalPayments        0
LoanCurrentDaysDelinquent           0
MonthlyLoanPayment                  0
EmploymentStatus                    0
date_numeric                    31237
dtype: int64

In [56]:
df_1.drop(columns=['date_numeric'], inplace=True)

In [57]:
df_1.to_csv('assets/model_data.csv', index=False)

In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Lasso
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Load the dataset
df = pd.read_csv('assets/model_data.csv')

# Identify categorical columns
categorical_cols = ['LoanOriginationQuarter', 'EmploymentStatus']

# Encode categorical features using LabelEncoder
le = LabelEncoder()
df[categorical_cols] = df[categorical_cols].apply(le.fit_transform)

# Split data into features and targets
X = df.drop(columns=['LoanStatus', 'ELA'])
y_class = df['LoanStatus']
y_reg = df['ELA']

# Split into training and testing sets
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X, y_class, y_reg, test_size=0.2, random_state=42
)

class CustomPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, classifiers, regressors):
        self.classifiers = classifiers
        self.regressors = regressors
        self.scaler = StandardScaler()
        self.encoders = {}
        self.categorical_cols = []  # Initialize as an empty list

    def fit(self, X, y_class, y_reg):
        # Identify categorical columns
        self.categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

        # Fit LabelEncoder for each categorical column
        for col in self.categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            self.encoders[col] = le

        # Scale the features
        X_scaled = self.scaler.fit_transform(X)

        self.fitted_classifiers = []
        self.fitted_regressors = []

        # Fit classifiers
        for clf in self.classifiers:
            clf.fit(X_scaled, y_class)
            self.fitted_classifiers.append(clf)

        # Fit regressors on filtered data
        for reg in self.regressors:
            X_filtered = X_scaled[y_class == 1]
            y_reg_filtered = y_reg[y_class == 1]
            reg.fit(X_filtered, y_reg_filtered)
            self.fitted_regressors.append(reg)

        return self

    def predict(self, X):
        # Encode categorical features
        for col in self.categorical_cols:
            if col in self.encoders:
                le = self.encoders[col]
                X[col] = le.transform(X[col])
            else:
                raise ValueError(f"Column '{col}' was not seen during fitting.")

        # Scale the features
        X_scaled = self.scaler.transform(X)

        # Predict class
        y_class_preds = np.array([clf.predict(X_scaled) for clf in self.fitted_classifiers])
        y_class_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=y_class_preds)  # Majority voting

        # Predict regression
        X_filtered = X_scaled[y_class_pred == 1]
        if len(X_filtered) == 0:
            return y_class_pred, np.array([])  # No samples for regression

        y_reg_preds = np.array([reg.predict(X_filtered) for reg in self.fitted_regressors])
        y_reg_pred = y_reg_preds.mean(axis=0)  # Averaging for regression

        return y_class_pred, y_reg_pred

# Initialize models
rfc = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
lr = LinearRegression()
lasso = Lasso(random_state=42, max_iter=10000, alpha=1.0)

classifiers = [rfc, xgb]
regressors = [lr, lasso]

# Create and fit the custom pipeline
pipeline = CustomPipeline(classifiers=classifiers, regressors=regressors)
pipeline.fit(X_train, y_class_train, y_reg_train)

# Make predictions
y_class_pred, y_reg_pred = pipeline.predict(X_test)
print("Classification Predictions:", y_class_pred)
print("Regression Predictions:", y_reg_pred)

joblib.dump(pipeline, 'assets/combined_pipeline.pkl')

Classification Predictions: [0 0 0 ... 0 0 0]
Regression Predictions: [ 96574.62370799 153615.3317471   60712.76680357 ...   8403.70010853
  34819.35512971 150722.80636579]


['assets/combined_pipeline.pkl']

In [93]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import pandas as pd

class CustomPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, classifiers, regressors):
        self.classifiers = classifiers
        self.regressors = regressors
        self.scaler = StandardScaler()
        self.encoders = {}
        self.categorical_cols = []

    def fit(self, X, y_class, y_reg):
        # Identify categorical columns
        self.categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

        # Fit LabelEncoder for each categorical column
        for col in self.categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            self.encoders[col] = le

        # Scale the features
        X_scaled = self.scaler.fit_transform(X)

        self.fitted_classifiers = []
        self.fitted_regressors = []

        # Fit classifiers
        for clf in self.classifiers:
            clf.fit(X_scaled, y_class)
            self.fitted_classifiers.append(clf)

        # Fit regressors on filtered data
        for reg in self.regressors:
            X_filtered = X_scaled[y_class == 1]
            y_reg_filtered = y_reg[y_class == 1]
            reg.fit(X_filtered, y_reg_filtered)
            self.fitted_regressors.append(reg)

        return self

    def predict(self, X):
        # Encode categorical features
        for col in self.categorical_cols:
            if col in self.encoders:
                le = self.encoders[col]
                X[col] = le.transform(X[col])
            else:
                raise ValueError(f"Column '{col}' was not seen during fitting.")

        # Scale the features
        X_scaled = self.scaler.transform(X)

        # Predict class
        y_class_preds = np.array([clf.predict(X_scaled) for clf in self.fitted_classifiers])
        y_class_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=y_class_preds)  # Majority voting

        # Predict regression
        X_filtered = X_scaled[y_class_pred == 1]
        if len(X_filtered) == 0:
            return y_class_pred, np.array([])  # No samples for regression

        y_reg_preds = np.array([reg.predict(X_filtered) for reg in self.fitted_regressors])
        y_reg_pred = y_reg_preds.mean(axis=0)  # Averaging for regression

        return y_class_pred, y_reg_pred

# Initialize models
rfc = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
lr = LinearRegression()
lasso = Lasso(random_state=42, max_iter=10000, alpha=1.0)

classifiers = [rfc, xgb]
regressors = [lr, lasso]

# Load the dataset
df = pd.read_csv('assets/model_data.csv')

# Identify categorical columns
categorical_cols = ['LoanOriginationQuarter', 'EmploymentStatus']

# Split data into features and targets
X = df.drop(columns=['LoanStatus', 'ELA'])
y_class = df['LoanStatus']
y_reg = df['ELA']

# Split into training and testing sets
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X, y_class, y_reg, test_size=0.2, random_state=42
)

# Create and fit the custom pipeline
pipeline = CustomPipeline(classifiers=classifiers, regressors=regressors)
pipeline.fit(X_train, y_class_train, y_reg_train)

# Save the pipeline using joblib
import joblib
joblib.dump(pipeline, 'assets/combined_pipeline.pkl')

['assets/combined_pipeline.pkl']

In [94]:
# Load the pipeline for prediction
pipeline = joblib.load('assets/combined_pipeline.pkl')

# Prepare new data
data = {
    'ListingNumber': [1074836],
    'BorrowerAPR': [0.30748],
    'BorrowerRate': [0.0920],
    'LoanMonthsSinceOrigination': [16],
    'LoanNumber': [102670],
    'LoanOriginationQuarter': ['Q1 2014'],
    'LP_CustomerPayments': [2819],
    'LP_CustomerPrincipalPayments': [1563],
    'LoanCurrentDaysDelinquent': [0],
    'MonthlyLoanPayment': [342],
    'EmploymentStatus': ['Employed']
}
new_data = pd.DataFrame(data)

# Make predictions
y_class_pred, y_reg_pred = pipeline.predict(new_data)

print("Classification Prediction:", y_class_pred)
print("Regression Prediction:", y_reg_pred)

Classification Prediction: [0]
Regression Prediction: []
